# Libraries

In [1]:
import bw2data as bd
import numpy as np

In [2]:
# Alias np.NaN to np.nan for backward compatibility
np.NaN = np.nan

# 1. Support Functions

In [4]:
from database_setup import results_to_dataframe
from lifecycle import calculate_impacts_for_activities
from data_parsing import combine_csvs_into_excel, combine_csvs_in_order

from config import recipe_midpoint_h_premise_gwp
from config import activities_li, reference_product_lithium, activities_ni, reference_product_nickel, activities_mn, reference_product_manganese
from config import activities_nmcoxide, reference_product_nmcoxide

# 2. Imports and Declarations

In [5]:
project_name = "LNV-EI38-20250414"

In [6]:
#Creating/accessing the project
bd.projects.set_current(project_name)

In [7]:
# We'll be performing the contribution analysis on our original databse
db_name = 'ecoinvent 3.8 cutoff'

# 3. Individual EFs

In [8]:
'''

The next section of the code allows to generate and export a contribution analysis in any activities we want.

The following segment (section 4), will solely focus on the activities we already decided on previously.

'''

'\n\nThe next section of the code allows to generate and export a contribution analysis in any activities we want.\n\nThe following segment (section 4), will solely focus on the activities we already decided on previously.\n\n'

In [ ]:
activities_of_interest = [
    # ('nickel mine operation and benefication to nickel concentrate, 7% Ni','CN')
    #('platinum group metal, extraction and refinery operations', 'ZA')
    # ('platinum group metal, mine and concentration operations', 'ZA')
    # ('processing of nickel-rich materials', 'GLO')
    # ('smelting and refining of nickel concentrate, 16% Ni', 'GLO')
    # ('nickel mine operation and benefication to nickel concentrate, 16% Ni', 'CA-QC')
    # ('cobalt production', 'GLO')
    # ('spodumene production','RoW')
    # ('lithium brine inspissation','GLO')
    # ('lithium carbonate production, from concentrated brine','GLO')
    # ('manganese(III) oxide production','CN')
    # ('manganese sulfate production','GLO')
    # ('manganese concentrate production','GLO')
]

# reference_product_nickel_mining = 'manganese concentrate'

#'platinum group metal concentrate'
# 'nickel, class 1'


In [ ]:
# results = calculate_impacts_for_activities(activities_of_interest, recipe_midpoint_h, db_name, reference_product_nickel)
# df = results_to_dataframe(results, project_name, db_name)

In [ ]:
# file_name = 'activities_name_here'
# df.to_csv(f'{file_name}.csv')

# 4. Elementary Flows from Ni, Mn, Li activities

## 4.1. Nickel

In [9]:
ni_file_name = 'EFcontributions_nickel.csv'

In [10]:
results = calculate_impacts_for_activities(activities_ni, recipe_midpoint_h_premise_gwp, db_name, reference_product_nickel)


 -- Calculating impacts for activity 'treatment of metal part of electronics scrap, in copper, anode, by electrolytic refining' in location 'RoW' using method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')'...

 ---- Top impacts for activity 'treatment of metal part of electronics scrap, in copper, anode, by electrolytic refining' in location 'RoW' and method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')':
Exchange: market for metal part of electronics scrap, in copper, anode, Type: technosphere, Compartment: None, Impact: 5.341118805007399e-05
Exchange: market for sulfuric acid, Type: technosphere, Compartment: None, Impact: 2.8521607589205628e-06
Exchange: market group for electricity, high voltage, Type: technosphere, Compartment: None, Impact: 2.1045474568881506e-06
Exchange: Copper, Type: biosphere, Compartment: ('air', 'non-urban air or from high stacks'), Impact: 2.0973318277366393e-06
Exchange: market group for electricity, high voltage, Type: 

In [11]:
df = results_to_dataframe(results, project_name, db_name)

In [12]:
df.to_csv(f'./contribution_analysis/{ni_file_name}')

## 4.2. Lithium

In [13]:
li_file_name = 'EFcontributions_lithium.csv'

In [14]:
# Call the function
results = calculate_impacts_for_activities(activities_li, recipe_midpoint_h_premise_gwp, db_name)


 -- Calculating impacts for activity 'spodumene production' in location 'AU' using method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')'...

 ---- Top impacts for activity 'spodumene production' in location 'AU' and method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')':
Exchange: market for electricity, medium voltage, Type: technosphere, Compartment: None, Impact: 6.162856107395309e-07
Exchange: market for heat, district or industrial, other than natural gas, Type: technosphere, Compartment: None, Impact: 5.659805800154213e-07
Exchange: market for industrial machine, heavy, unspecified, Type: technosphere, Compartment: None, Impact: 2.5100118191554177e-07
Exchange: market for blasting, Type: technosphere, Compartment: None, Impact: 1.0571906335177184e-07
Exchange: market for diesel, burned in building machine, Type: technosphere, Compartment: None, Impact: 9.76687913363623e-08

 -- Calculating impacts for activity 'spodumene production' in location '

In [15]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [16]:
df.to_csv(f'./contribution_analysis/{li_file_name}')

## 4.3. Manganese

In [20]:
mn_file_name = 'EFcontributions_manganese.csv'

In [17]:
# Call the function
results = calculate_impacts_for_activities(activities_mn, recipe_midpoint_h_premise_gwp, db_name, reference_product_manganese)


 -- Calculating impacts for activity 'manganese concentrate production' in location 'GLO' using method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')'...

 ---- Top impacts for activity 'manganese concentrate production' in location 'GLO' and method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')':
Exchange: market for sulfur dioxide, liquid, Type: technosphere, Compartment: None, Impact: 3.4412147054759464e-05
Exchange: market for chemical factory, organics, Type: technosphere, Compartment: None, Impact: 1.653628855947904e-05
Exchange: market for manganese concentrate, Type: technosphere, Compartment: None, Impact: 1.5951746537064253e-05
Exchange: market group for electricity, medium voltage, Type: technosphere, Compartment: None, Impact: 1.1185784154806591e-05
Exchange: market for sulfur dioxide, liquid, Type: technosphere, Compartment: None, Impact: 5.6211532139467e-06

 -- Calculating impacts for activity 'manganese concentrate production' in locatio

In [18]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [21]:
df.to_csv(f'./contribution_analysis/{mn_file_name}')

## 4.4. NMC oxide

In [22]:
# Call the function
results = calculate_impacts_for_activities(activities_nmcoxide, recipe_midpoint_h_premise_gwp, db_name)


 -- Calculating impacts for activity 'NMC111 oxide production, for Li-ion battery' in location 'CN' using method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')'...

 ---- Top impacts for activity 'NMC111 oxide production, for Li-ion battery' in location 'CN' and method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')':
Exchange: market for NMC111 hydroxide, Type: technosphere, Compartment: None, Impact: 0.01511517701516304
Exchange: market for lithium carbonate, Type: technosphere, Compartment: None, Impact: 0.00034430378084616807
Exchange: market group for electricity, medium voltage, Type: technosphere, Compartment: None, Impact: 0.0002653763482807883
Exchange: market for chemical factory, organics, Type: technosphere, Compartment: None, Impact: 3.0633474556435045e-05
Exchange: Carbon dioxide, fossil, Type: biosphere, Compartment: ('air',), Impact: 0.0

 -- Calculating impacts for activity 'NMC111 oxide production, for Li-ion battery' in location 'CN' u

In [23]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [29]:
df.to_csv('./contribution_analysis/EFcontributions_nmcoxide.csv')

In [25]:
# We'll run these again post activities' modification

# 5. Organising the Excel to be analysed

In [26]:
###
# Right now I haven't figured the best way to do this, but here it goes:
# I'm breaking down the contribution analysis per:
## 1. Subsystem (e.g. Nickel )
## 2. Activities in that subsystem

In [27]:
# Combine the CSVs into a single Excel file
folder = "contribution_analysis"
# Proof of concept, improve on the code:
output_xlsx = "processes_and_contributions.xlsx"

csv_files = [
    ni_file_name,
    li_file_name,
    mn_file_name,
]

combine_csvs_in_order(
    folder_path=folder,
    csv_list=csv_files,
    output_file=output_xlsx,
    activity_col="activity_id"
)

Excel file saved as: processes_and_contributions.xlsx


In [28]:
# A dictionary mapping (database, activity_uuid) --> short label
ACTIVITY_ID_MAP = {
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "7b370e739f88f9729064638bf7210c78"): "2",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "9f9aaa855cf07859c28cb6fc84e3a2fd"): "3_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "bde92d093e012fc74b4cf23bd73869be"): "3_B",
    # WARNING: Next line has the same tuple as the one two lines below but different labels
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "ecc38c81906feff810d89bb6c123560a"): "4_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "f5ec689f258fc22994162fe53b025b0d"): "4_B",
    # The following line overwrites the key ("EI38_cutoff_remind_SSP1-Base_2025_baseline","ecc38c81906feff810d89bb6c123560a"):
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "ecc38c81906feff810d89bb6c123560a"): "5_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "b2ab8da844b26789a68138f5eebc21eb"): "5_B",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "fb970e38e35ff930ea6ca1865d33cc52"): "6_A",
    ("EI38_cutoff_remind_SSP1-Base_2020_energy",   "3dd23e526c5de9474538ce20d43a402f"): "6_B",

    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "d0953092b5b10c411f947eb33b8f4add"): "8",

    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "31bda81a4c4a3a273f827fc8832f4da3"): "9_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "0b5870b08f744c37b54945c34adb7dcd"): "9_B",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "adfb686ff68d3eac8aaa5ea0cb3c169e"): "9_C",
    # Overwrites 9_B if the first line was repeated, etc.
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "31bda81a4c4a3a273f827fc8832f4da3"): "10_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "0b5870b08f744c37b54945c34adb7dcd"): "10_B",
    ("ecoinvent 3.8 cutoff",                       "25a7644f55c4c4c890356f515cb671fa"): "10_C",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "d0617f6353f60771cfb7dcdfe6c78ee1"): "11_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "4bcdd0870049fd3b15b4cfa1b4ed580f"): "11_B",

    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "52112ff78d75aabf60bd4b63055d0a61"): "13_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "dc75ef318dcef071d7f6c578aba30921"): "13_B",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "3d908539006e39885946109381e53214"): "13_C",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "4c6809146c7ded41c88cfcb01111a595"): "13_D",
    # Overwrites 13_A if repeated, etc.
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "52112ff78d75aabf60bd4b63055d0a61"): "14_A",
    ("EI38_cutoff_remind_SSP1-Base_2025_baseline", "ac80cf413845bda6587f576899e27405"): "14_B",
}
